<a href="https://colab.research.google.com/github/iqbalfikdata/Statistika/blob/master/Statistik_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#install researchpy
!pip install researchpy

In [0]:
#yang kita butuhkan
import numpy as np, pandas as pd, researchpy as rp
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import MultiComparison, pairwise_tukeyhsd
from scipy import stats
from itertools import combinations

**1. Melakukan Uji Statistik Untuk Dua Grup**

In [0]:
# Membuat helper-function untuk medapatkan hasil statistik dari dua buah grup
def uji_2grup(x1, x2, berpasangan=False):
  normal, Pnormal = stats.shapiro(x1 - x2)     #uji normalitas
  levene, Plevene = stats.levene(x1, x2)       #uji homogenitas
  if Pnormal < .05 and Plevene > .05 and berpasangan==False:      #uji non parametrik MannWhitenyU
    T, pValue = stats.mannwhitneyu(x1, x2)
    if pValue > .05:
      print('Uji MannWhitneyU menghasilkan pValue {}, tidak ada perbedaan yang nyata antara 2 grup tsb'.format(pValue))
    else:
      print('Uji MannWhitneyU menghasilkan pValue {}, ada perbedaan yang signifikan anatara 2 grup tsb'.format(pValue))
  elif Pnormal > .05 and Plevene < .05 and berpasangan==False:       #uji welch
    desc, hasil = rp.ttest(x1, x2, equal_variances=False)
    print(desc)
    print(hasil)
  elif Pnormal > .05 and Plevene > .05 and berpasangan==False:       #uji T saling bebas
    desc, hasil = rp.ttest(x1, x2)
    print(desc)
    print(hasil)
  elif Pnormal > .05 and berpasangan == True:          #uji T yang berpasangan
    desc, hasil = rp.ttest(x1, x2, paired=True)
    print(desc)
    print(hasil)
  elif Pnormal < .05 and berpasangan == True:         #uji non paramatrik Wilcoxon
    print(rp.ttest(x1, x2, paired=True, equal_variances=False))

In [5]:
#misalkan ada dua produk saus, saus A dan saus B, masing-masing diambil 6 sample kemudian
#diukur tingkat kepedasannya (dengan satuan tertentu), kita ingin tahu apakah kedua saus itu
#memiliki tingkat kepedasan yang berbeda secara signifikan

x1 = np.array([30, 40, 20, 60, 20, 50])
x2 = np.array([90, 70, 50, 100, 60, 80])
df = pd.DataFrame({'saus_A':x1, 'saus_B':x2})
df.head(3)

,saus_A,saus_B
0,30,90
1,40,70
2,20,50


In [6]:
uji_2grup(df.saus_A, df.saus_B)

Uji MannWhitneyU menghasilkan pValue 0.006295955745189842, ada perbedaan yang signifikan anatara 2 grup tsb


In [0]:
#Well, ternyata apa yang baru kita lakukan termasuk uji Mann WhitneyU, dan hasilnya adalah bahwa
#saus A dam saus B memiliki rasa pedas yang berbeda secara signifikan (tolak H0)
pass

In [8]:
#misalkan seorang dosen menggunakan metode mengajar X terhadap 22 mahasiswa, lalu ia mencatat skor
#hasil belajar matematika mereka antara sebelum dan sesudah menggunakan metode X tadi, ia ingin
#mengetahui apakah metode tsb berhasil secara signifikan memberi perubahan terhadap hasil belajarnya

x3 = [56, 72, 67, 58, 70, 68, 76, 70, 69, 58, 65, 70, 75, 67, 72, 74, 76, 68,
     62, 70, 61, 77]
x4 = [87, 92, 87, 62, 89, 86, 90, 86, 80, 85, 90, 83, 80, 87, 82, 80, 83, 82,
     89, 85, 92, 85]
df1 = pd.DataFrame({'pre':x3, 'post':x4})
df1.tail()

,pre,post
17,68,82
18,62,89
19,70,85
20,61,92
21,77,85


In [9]:
uji_2grup(df1.pre, df1.post, berpasangan=True)

  Variable     N       Mean        SD        SE  95% Conf.   Interval
0      pre  22.0  68.227273  6.101770  1.300902  65.521899  70.932646
1     post  22.0  84.636364  6.252965  1.333137  81.863954  87.408773
2     diff  22.0 -16.409091  8.221106  1.752746 -20.054125 -12.764057
        Paired samples t-test  results
0  Difference (pre - post) =  -16.4091
1       Degrees of freedom =   21.0000
2                        t =   -9.3619
3    Two side test p value =    0.0000
4   Difference < 0 p value =    0.0000
5   Difference > 0 p value =    1.0000
6                Cohen's d =   -1.9960
7                Hedge's g =   -1.9601
8            Glass's delta =   -2.6892
9                        r =    0.8982


In [0]:
#dari table diatas, kita lihat bahwa Pvalue nya relatif kecil sekali (<0,05), artinya kita bisa
#menerima H1, yaitu bahwa metode mengajar X memang bisa memberikan hasil belajar yang berbeda
#secara signifikan

#dengan menggunakan researchpy, banyak informasi berguna yang kita dapatkan dari pada menggunkan
#scipy, dari tabel diatas bisa kita lihat nilai Cohen's d dan Hedge's g yang merupakan ukuran
#kekuatan efek(seberapa jauh 2 grup diatas berbeda), untuk nilai yang > 0,8 dikatan bahwa kekuatan
#efek nya lumayan besar, sehingga dapat kita katakan bahwa perbedaan dari kedua hasi tes belajar mereka
#lumayan besar (tanda negatif pada nilai ini abaikan saja)

#Kesimpulannya: Metode mengajar X membawa efek yang lumayan besar dalam membuat perbedaan hasil
#belajar mahasiswa
pass

In [11]:
#seorang menejer suatu perusahaan mengundang Trainer untuk memotivasi karyawannya, ia meminta
#seorang statistik untuk mengetahui apakah Trainer tsb bisa memberi perbedaan yang nyata terhadap
#produktivitas karyawannya, lalu sang menejer memberikan data nilai produktivitas karyawan antara
#sebelum dan sesudah mengundang Trainer (dalam satuan tertentu)

x7 = np.array([67, 84, 76, 34, 31, 79, 88, 47, 35, 41, 34, 37, 32, 26, 90])
x8 = np.array([75, 90, 88, 42, 47, 82, 77, 55, 37, 50, 44, 46, 46, 38, 93])
df2 = pd.DataFrame({'sebelum_training':x7, 'sesudah_training':x8})
df2.sample(5, random_state=2020)

,sebelum_training,sesudah_training
9,41,50
1,84,90
2,76,88
4,31,47
11,37,46


In [12]:
uji_2grup(df2.sebelum_training, df2.sesudah_training, berpasangan=True)

      Wilcoxon signed-rank test    results
0  Mean for sebelum_training =   53.400000
1  Mean for sesudah_training =   60.666667
2                    T value =   11.000000
3                    Z value =   -2.783000
4          Two sided p value =    0.005800
5                          r =   -0.508100


In [0]:
#Ternyata ini termasuk dalam uji Wilcoxon, dari sana kita lihat bahwa nilai Pvalue nya < 0,05
#Berarti Trainer tsb berhasil membawa perbedaan yang nyata terhadap produktivitas karyawan disana
#Terima H1
pass

In [14]:
#Berikut disajikan kekuatan durasi nyala lampu (dalam hari)
#apakah ada perbedaan rata-rata nyala lampu P dengan nyala lampu Q secara signifikan?

x5 = np.array([63, 54, 72, 63, 45, 54, 45, 54, 54, 63, 54, 54])
x6 = np.array([72, 54, 63, 72, 54, 54, 62, 65, 70, 62, 55, 63])
df3 = pd.DataFrame({'lampu_P':x5, 'lampu_Q':x6})
df3.head()

,lampu_P,lampu_Q
0,63,72
1,54,54
2,72,63
3,63,72
4,45,54


In [15]:
uji_2grup(df3.lampu_P, df3.lampu_Q)

   Variable     N       Mean        SD        SE  95% Conf.   Interval
0   lampu_P  12.0  56.250000  7.794229  2.250000  51.297783  61.202217
1   lampu_Q  12.0  62.166667  6.846809  1.976503  57.816412  66.516921
2  combined  24.0  59.208333  7.785042  1.589115  55.920998  62.495668
                  Independent t-test  results
0  Difference (lampu_P - lampu_Q) =   -5.9167
1              Degrees of freedom =   22.0000
2                               t =   -1.9756
3           Two side test p value =    0.0609
4          Difference < 0 p value =    0.0304
5          Difference > 0 p value =    0.9696
6                       Cohen's d =   -0.8065
7                       Hedge's g =   -0.7787
8                   Glass's delta =   -0.7591
9                               r =    0.3882


In [0]:
#Termasuk uji T saling bebas, p value nya > 0,05, artinya kita gagal menolak H0
#kesimpulannya: tidak ada perbedaan yang berarti antara nyala lampu merek P dengan nyala lampu merek Q
pass

**2. Melakukan Uji Frekuensi**

In [0]:
#membuat helper-function
def uji_frek(x, berpasangan=False):
  if  x.ndim ==1 and berpasangan==False:
    chi, p = stats.chisquare(x)
    if p > .05:
      print('karena nilai p = {}, kejadian ini terjadi secara alami'.format(p))
    else: print('nilai p = {}, ada faktor luar yang membuat kejadian ini bisa terjadi'.format(p))
  elif x.ndim ==2 and berpasangan==False:
    chi, p, dof, ex = stats.chi2_contingency(x)
    if p > 0.05:
      print('uji chi2 kontingensi p = {}, ini terjadi secara alami'.format(p))
    else: print('uji chi2 kontingensi p = {} , ada faktor luar yang membuat hal ini bisa terjadi'.format(p))
  elif x.ndim ==2 and berpasangan==True:
    tabel, hasil = rp.crosstab(x.iloc[:,0], x.iloc[:,1], test='mcnemar')
    print(tabel)
    print(hasil)

In [18]:
#seorang menejer perusahaan sebuah pabrik kartu olahraga merencanakan membuat seri kartu dengan
#gambar pemain sepakbola terkenal. Pemain tersebut adalah Messi, Ronaldo, Sneidjer, Torres, Xavi
#dan Gerrard. Di hari terakhir ia behasil menjual 120 kartu dengan sebaran sebagai berikut

pemain = ['Messi', 'Ronaldo', 'Sneidjer', 'Torres', 'Xavi', 'Gerrard']
terjual = [13, 33, 14, 7, 36, 17]
data = pd.DataFrame({'pemain':pemain, 'terjual':terjual}).set_index('pemain')
data

,terjual
pemain,
Messi,13
Ronaldo,33
Sneidjer,14
Torres,7
Xavi,36
Gerrard,17


In [19]:
#dari tabel diatas, kita ingin tahu apakah sebaran itu terjadi secara alami atau ada faktor luar
#dalam hal ini gambar pemain menentukan sebaran terjualnya kartu ?

uji_frek(data.terjual, berpasangan=False)

nilai p = 1.9819328179330533e-06, ada faktor luar yang membuat kejadian ini bisa terjadi


In [20]:
#Sekarang, kita akan meneliti apakah jenis kelamin menentukan partai yang akan dipilih

data1 = pd.DataFrame({'partai_A':[26, 20], 'partai_B':[13, 29], 'partai_C':[5, 7],
                 'jenis_kelamin':['laki-laki', 'perempuan']}).set_index('jenis_kelamin')
data1

,partai_A,partai_B,partai_C
jenis_kelamin,,,
laki-laki,26,13,5
perempuan,20,29,7


In [21]:
uji_frek(data1, berpasangan=False)

uji chi2 kontingensi p = 0.05351733509077522, ini terjadi secara alami


In [0]:
#Karena nilai P value nya > 0,05, kejadiannya terjadi secara alami atau dalam pemilihan sebuah partai
#itu tidak ditentukan oleh jenis kelamin (gagal menolak H0)
pass

In [24]:
#Seorang peneliti ingin mengetahui apakah tehnik marketing X dapat membuat pembeli mau membeli suatu
#produk, diambillah sampel pembeli sebanyak 80 orang dan dicatat apakah ia mau membeli atau tidak setelah
#dan sebelum tehnik marketing X dilakukan, berikut datanya

sebelum = np.random.randint(0, 2, 80)
sesudah = np.random.randint(0, 2, 80)
data2 = pd.DataFrame({'sebelum':sebelum, 'sesudah':sesudah})
data2.sample(5)

,sebelum,sesudah
72,0,0
51,1,0
29,1,0
76,0,1
78,1,0


In [25]:
#0 = tidak mau membeli, 1=mau membeli
uji_frek(data2, berpasangan=True)

        sesudah        
              0   1 All
sebelum                
0            16  21  37
1            23  20  43
All          39  41  80
                          McNemar  results
0  McNemar's Chi-square ( 1.0) =    0.0909
1                      p-value =    0.7630
2                 Cramer's phi =    0.0337


In [0]:
#Tenyata kita dapatkan bahwa p value nya = 0.34, karena nilainya > 0,05 maka dapat dikatakan bahwa
#kita gagal menolak H0, yang berarti metode marketing X tidak mempengaruhi mau atau tidak nya pembeli
#atas produk yang ditawarkan
pass

**3. Uji Statistik Untuk Lebih Dari Dua Grup**

In [29]:
#Berikut disajikan data jumlah penjualan (dalam unit) hp merek samsung, oppo, vivo, lenovo
#di hari terakhir

#Empat hal yang ingin kita jawab adalah:
#1. Apakah merek hp mempengaruhi hasil penjualan ?
#2. Jika mempengaruhi, seberapa besar efeknya ?
#3. Apakah ada perbedaan rata-rata penjualan antara keempat merek hp tsb ?
#4. Merek manakah yang rata-rata nya berbeda secara signifikan ?

aov = pd.read_csv('/content/anova_one.csv')
aov.head()

,Unnamed: 0,samsung,oppo,vivo,lenovo
0,0,52,48,34,32
1,1,45,36,32,33
2,2,48,33,37,36
3,3,36,38,35,39
4,4,39,49,42,38


In [30]:
#sebelumnya, kita akan modifikasi tabel diatas sehingga merek hp menjadi variabel prediktor
#dan hasil penjualannya menjadi variabel target

aov = pd.melt(aov, id_vars='Unnamed: 0', var_name='Merek', value_name='penjualan')
aov.sample(5)

,Unnamed: 0,Merek,penjualan
20,0,vivo,34
18,8,oppo,40
2,2,samsung,48
35,5,lenovo,35
22,2,vivo,37


In [31]:
#melakukan analisis varians dengan menggunakan statsmodels

model = ols('penjualan ~ C(Merek)', data=aov).fit()
aov_df = anova_lm(model)
print(model.summary())
print(aov_df)

                            OLS Regression Results                            
Dep. Variable:              penjualan   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     4.445
Date:                Sat, 28 Mar 2020   Prob (F-statistic):            0.00933
Time:                        00:59:26   Log-Likelihood:                -126.52
No. Observations:                  40   AIC:                             261.0
Df Residuals:                      36   BIC:                             267.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              35.7000    

In [32]:
#Interpretation time!
#jawaban dari nomer 1 dan 3 adalah: Ya, merek HP mempengaruhi hasil penjualan, ini bisa dilihat dari
#nilai prob (F-statistik) = 0,009, yang kurang dari 0,05. Artinya kita terima H1, yaitu ada perbedaan
#rata-rata penjualan yang nyata antara salah satu atau beberapa kelompok dari merek HP tersebut
#untuk menjawab no 4, kelompok mana yang yang berbeda, kita akan lakukan uji post-hoc

post_hoc = MultiComparison(aov.penjualan, aov.Merek).tukeyhsd()
print(post_hoc)

  Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1  group2 meandiff p-adj   lower    upper  reject
-------------------------------------------------------
 lenovo    oppo      5.4 0.2061  -1.8632 12.6632  False
 lenovo samsung      9.6 0.0056   2.3368 16.8632   True
 lenovo    vivo      3.3 0.6042  -3.9632 10.5632  False
   oppo samsung      4.2 0.4165  -3.0632 11.4632  False
   oppo    vivo     -2.1 0.8499  -9.3632  5.1632  False
samsung    vivo     -6.3 0.1088 -13.5632  0.9632  False
-------------------------------------------------------


In [0]:
#Wah ternyata rata-rata penjualan yang berbeda signifikan itu jatuh pada merek samsung dan lenovo
#dilihat dari kolom reject untuk lenovo dan samsung adalah True, artinya kita menolak H0
#Terakhir kita akan mengukur kekuatan efek pengaruh merek HP terhadap hasil penjualan dengan
#menggunakan rumus dibawah
pass

$ \frac{SS_M - df_M\times MS_R}{SS_T + MS_R} $

In [34]:
SSM, dfM = aov_df['sum_sq'][:-1], aov_df['df'][0]
MSR, SST = aov_df['mean_sq'][-1], sum(aov_df['sum_sq'])
pembilang = SSM - dfM*MSR
penyebut = SST + MSR
aov_df['omega'] = pembilang / penyebut
aov_df

,df,sum_sq,mean_sq,F,PR(>F),omega
C(Merek),3.0,484.875,161.625000,4.444657,0.009329,0.205308
Residual,36.0,1309.100,36.363889,NaN,NaN,NaN


In [0]:
#dari tabel diatas, karena nilai omega yang kita dapatkan .2, maka efek yang diberikan merek HP
#terhadap hasil penjualan dapat dikatakan sangat lemah
pass

In [57]:
#Selanjutnya kita akan menambah jumlah prediktor (anova 2 jalur)
#berikut disajikan tabel nya

aov2 = pd.read_csv('/content/aov_2jalur.csv')
aov2.head()

,Unnamed: 0,Unnamed: 0.1,Jenis_kelamin,masa_kerja,Gaji
0,0,2,1,1,2.5
1,1,3,1,2,3.2
2,2,4,1,2,3.7
3,3,5,1,3,2.2
4,4,6,1,2,2.5


In [0]:
#untuk kolom jenis kelamin, 0 = female, 1 = male, satuan masa kerja dalam tahun

#Hal-hal yang ingin kita tanyakan dalam tabel diatas adalah
#1. Apakah jenis kelamin mempengaruhi gaji,  
#2. Apakah masa kerja mempengaruhi gaji
#3. Apakah interaksi antara jenis kelamin dan masa kerja mempengaruhi besaran gaji yang didapat?

#kita akan drop dulu kolom Unnamed: 0 dan Unnamed: 0.1, karena kita tidak membutuhannya dalam analisis
aov2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [61]:
#kita akan mengetes faktor jenis kelamin, masa kerja dan interaksinya terhadap gai
model = ols('Gaji ~ C(Jenis_kelamin)*C(masa_kerja) ',data=aov2).fit()
aov2_df = anova_lm(model)
print(model.summary())
print(aov2_df)

                            OLS Regression Results                            
Dep. Variable:                   Gaji   R-squared:                       0.663
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                     15.35
Date:                Sat, 28 Mar 2020   Prob (F-statistic):           2.45e-08
Time:                        02:17:20   Log-Likelihood:                -23.119
No. Observations:                  45   AIC:                             58.24
Df Residuals:                      39   BIC:                             69.08
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [64]:
#interpretation time!
#Secara keseluruhan, model yang kita miliki signifikan terhadap variable target (gaji), ini bisa
#dilihat dari nilai Pvalue = Prob (Fstat) pada tabel OLS < 0,05

#Kemudian di tabel kedua, nilai Pvalue untuk jenis kelamin > 0,05, yaitu 0,72, ini berarti bahwa
#jenis kelamin tidak mempengaruhi besaran gaji

#Juga nilai Pvalue untuk interaksi antara Jenis kelamin dan masa kerja > 0,05, ini berarti bahwa
#interaksi mereka tidak memberikan perbedaan yang signifikan terhadap besaran gaji

#Nah karena jenis kelamin dan interaksi jenis kelamin dengan masa kerja itu tidak signifikan, maka
#kita akan remove mereka dari model, kita akan menjalankan model baru tanpa mereka

model_new = ols('Gaji ~ C(masa_kerja)', data=aov2).fit()
aov2_df = anova_lm(model_new)
print(model_new.summary())
print(aov2_df)

                            OLS Regression Results                            
Dep. Variable:                   Gaji   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     35.64
Date:                Sat, 28 Mar 2020   Prob (F-statistic):           8.94e-10
Time:                        02:39:04   Log-Likelihood:                -25.269
No. Observations:                  45   AIC:                             56.54
Df Residuals:                      42   BIC:                             61.96
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              3.5333      0

In [0]:
#Wah ternyata model baru yang baru kita buat lebih baik dari model sebelumnya, ini bisa dilihat
#dari nilai AIC pada model baru kita lebih kecil dari nilai AIC pada model sebelumnya
#makin kecil nilai AIC, makin bagus suatu model

#Ok, jadi sampai disini, hanya masa kerja lah yang memberi efek terhadap besaran gaji

#Pertanyaan selanjutnya adalah, 
#1. Seberapa besar efeknya? 
#2. Mereka yang bekerja berapa tahunkah yang gajinya berbeda signifikan dengan yang lain?
pass

In [66]:
#Untuk menjawab soal no 2, kita bisa gunakan dua cara, yaitu tukeyhsd dan bonferroni test
#Cara 1, dengan tukeyhsd:
tukey = MultiComparison(aov2.Gaji, aov2.masa_kerja).tukeyhsd()
print(tukey)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     1      2  -0.3467 0.0897 -0.7362  0.0429  False
     1      3  -1.3067  0.001 -1.6962 -0.9171   True
     2      3    -0.96  0.001 -1.3496 -0.5704   True
----------------------------------------------------


In [0]:
#Well, ternyata yang besar gajinya berbeda secara nyata adalah antara mereka yang masa kerjanya
#1 tahun dan 3 tahun, juga antara mereka yang masa kerjanya 2 tahun dengan 3 tahun
pass

In [0]:
#Cara dua, Bonferroni test:
#Bonferroni test sama dengan uji t saling lepas untuk tiap dua grup, tetapi dengan Pvalue yang
#baru, Pvalue yang baru = Pvalue lama / banyak grup

new_pvalue = .05 / 3
def bonferroni(*args, **kwargs):
  data, Pvalue, grup = [], [], []
  for i in args:
    data.append(i)
  data = list(combinations(data, 2))
  for i in data:
    t_stat, t_pvalue = stats.ttest_ind(i[0], i[1])
    Pvalue.append(t_pvalue)
  for i in kwargs:
    grup.append(i)
  grup = list(combinations(grup, 2))

  bon_df = pd.DataFrame({'grup':grup, 'Pvalue':Pvalue})
  bon_df['Tolak'] = 'No'
  bon_df.loc[bon_df['Pvalue'] < new_pvalue, 'Tolak'] = 'yes'
  return bon_df

In [97]:
tahun_1, tahun_2 = aov2.Gaji[aov2['masa_kerja']==1], aov2.Gaji[aov2['masa_kerja']==2]
tahun_3 = aov2.Gaji[aov2['masa_kerja']==3]

bonferroni(tahun_1, tahun_2, tahun_3,
           tahun_1='grup1', tahun_2='grup2', tahun_3='grup3')

,grup,Pvalue,Tolak
0,"(tahun_1, tahun_2)",4.189084e-02,No
1,"(tahun_1, tahun_3)",9.703616e-10,yes
2,"(tahun_2, tahun_3)",5.636262e-06,yes


In [94]:
#Ok ok, dan bonferroni test dengan tukeyhsd sama saja ^_^

#Lebih lanjut, jika ingin menguji asumsi dari test ini, kita tinggal eksekusi kode dibawah
model_new.diagn

{'condno': 3.7320508075688776,
 'jb': 3.8325743432840165,
 'jbpv': 0.14715230036399457,
 'kurtosis': 4.145830086877002,
 'mineigval': 4.019237886466839,
 'omni': 4.699147977499592,
 'omnipv': 0.09540979920686912,
 'skew': 0.42752574827690737}

In [0]:
#condno = conditional number, ini adalah multikolinearitas test, ia memberi informasi apakah antar
#grup itu ada keterhubungan, kalau nilainya > 20, maka ada gejala multikolinearitas

#jb = Jarque-Bera dengan nilai Pvalue = jbpv = 0, 14, bahwa tiap grup mengikuti distribusi Gauss (normal)
#omni = Omnibus dengan nilai Pvalue = omnipv = 0,09, bahwa tiap grup homogen

#Selanjutnya akan kita ukur kekuatan efek masa kerja terhadap perbedaan gaji dengan rumus yang sama
#seperti sebelumnya
pass

In [99]:
SSM, dfM = aov2_df['sum_sq'][:-1], aov2_df['df'][0]
MSR, SST = aov2_df['mean_sq'][-1], sum(aov2_df['sum_sq'])
pembilang = SSM - dfM*MSR
penyebut = SST + MSR
aov2_df['omega'] = pembilang / penyebut
aov2_df

,df,sum_sq,mean_sq,F,PR(>F),omega
C(masa_kerja),2.0,13.745778,6.872889,35.637202,8.942679e-10,0.606211
Residual,42.0,8.100000,0.192857,NaN,NaN,NaN


In [0]:
#Untuk nilai omega yang terletak pada interval 0,5 sampai 0,8, kategorinya adalah sedang
#Jadi, intensitas efek masa_kerja terhadap gaji adalah sedang
pass

In [0]:
#Pada saat kita melakukan uji anova diatas, tiap pasang grup lulus uji normalitasnya, bagaimana jika
#salah satu dari pasangan grup tidak memenuhi uji normalitas?
#maka uji yang tepat kita lakukan adalah uji nonparametrik Kruskal-Wallis (jika tiap grup berpasangan)
#dan Friedman (jika tiap grup berpasangan)

#Sebagai contoh, disajikan tiga buah grup berikut
g1 = np.array([30, 40, 20, 60, 20, 50])
g2 = np.array([90, 70, 50, 100, 60, 80])
g3 = np.array([45, 65, 50, 85, 62, 83])

#kita akan cek apakah ada pasangan dari grup itu yang tidak mengikuti distribusi Gauss
#Cara dua, Bonferroni test:

def cek_normal(*args):
  data, Pvalue, = [], []
  for i in args:
    data.append(i)
  data = list(combinations(data, 2))
  for i in data:
    normal, Pnormal = stats.shapiro(i[0] - i[1])
    Pvalue.append(Pnormal)
  print(Pvalue)

In [129]:
cek_normal(g1, g2, g3)

[0.032937321811914444, 0.9359415173530579, 0.03174012526869774]


In [0]:
#Nah, karena ada dua pasang grup yang nilai Pvalue nya dibawah 0,05, maka uji nonparametrik
#Kruskal lebih cocok daripada Anova

#membua helper-function untuk uji nonparametrik lebih dari dua grup:
def nonparametrik_3(x1, x2, x3, berpasangan=False):
  if berpasangan==False:
    stat, Pvalue = stats.kruskal(x1, x2, x3)
    if Pvalue > .05:
      print(' [Uji Kruskal] Terima H0, tidak ada perbedaan yang nyata antar grup')
    else: print('[Uji Kruskal] Tolak H0, ada perbedaan yang signifikan antar grup')
  else:
    stat, Pvalue = stats.friedmanchisquare(x1, x2, x3)
    if Pvalue > .05:
      print('[Uji Friedman] Terima H0, tidak ada perbedaan yang signifikan antar grup')
    else: print('[Uji Friedman] Tolak H0, ada perbedaan yang berarti antar grup')  



In [134]:
#Melakukan Uji Kruskal dan Friedman
print(nonparametrik_3(g1, g2, g3))
print(nonparametrik_3(g1, g2, g3, berpasangan=True))

[Uji Kruskal] Tolak H0, ada perbedaan yang signifikan antar grup
None
[Uji Friedman] Tolak H0, ada perbedaan yang berarti antar grup
None


In [0]:
# Terima Kasih #